In [34]:
import dropbox
from dropbox.exceptions import ApiError, AuthError
import os
from pathlib import Path

class DropboxUploader:
    def __init__(self, access_token):
        """Initialize Dropbox client with access token"""
        self.access_token = access_token
        try:
            self.dbx = dropbox.Dropbox(self.access_token)
            # Verify the token works
            self.dbx.users_get_current_account()
            print("Successfully connected to Dropbox!")
        except AuthError:
            print("ERROR: Invalid access token. Please check your token.")
            exit(1)
    
    def upload_file(self, local_path, dropbox_path):
        """
        Upload a single file to Dropbox (handles large files automatically)
        
        Args:
            local_path: Path to the local file
            dropbox_path: Destination path in Dropbox (must start with /)
        """
        try:
            file_size = os.path.getsize(local_path)
            CHUNK_SIZE = 4 * 1024 * 1024  # 4MB chunks
            
            print(f"Uploading {local_path} ({file_size / (1024*1024):.2f} MB) to {dropbox_path}...")
            
            with open(local_path, 'rb') as f:
                # Use simple upload for small files
                if file_size <= 150 * 1024 * 1024:  # 150 MB
                    self.dbx.files_upload(
                        f.read(), 
                        dropbox_path, 
                        mode=dropbox.files.WriteMode.overwrite
                    )
                else:
                    # Use upload session for large files
                    upload_session_start_result = self.dbx.files_upload_session_start(
                        f.read(CHUNK_SIZE)
                    )
                    session_id = upload_session_start_result.session_id
                    cursor = dropbox.files.UploadSessionCursor(
                        session_id=session_id,
                        offset=f.tell()
                    )
                    commit = dropbox.files.CommitInfo(
                        path=dropbox_path,
                        mode=dropbox.files.WriteMode.overwrite
                    )
                    
                    # Upload chunks
                    while f.tell() < file_size:
                        if (file_size - f.tell()) <= CHUNK_SIZE:
                            # Last chunk
                            self.dbx.files_upload_session_finish(
                                f.read(CHUNK_SIZE),
                                cursor,
                                commit
                            )
                        else:
                            self.dbx.files_upload_session_append_v2(
                                f.read(CHUNK_SIZE),
                                cursor
                            )
                            cursor.offset = f.tell()
                        
                        # Progress indicator
                        progress = (f.tell() / file_size) * 100
                        print(f"  Progress: {progress:.1f}%", end='\r')
                    
                    print()  # New line after progress
            
            print(f"✓ Successfully uploaded {local_path}")
            return True
        except ApiError as e:
            print(f"✗ Error uploading {local_path}: {e}")
            return False
        except FileNotFoundError:
            print(f"✗ File not found: {local_path}")
            return False
    
    def upload_directory(self, local_dir, dropbox_dir):
        """
        Upload all files from a local directory to Dropbox
        
        Args:
            local_dir: Path to local directory
            dropbox_dir: Destination directory in Dropbox (must start with /)
        """
        local_path = Path(local_dir)
        
        if not local_path.exists():
            print(f"✗ Directory not found: {local_dir}")
            return
        
        if not local_path.is_dir():
            print(f"✗ Not a directory: {local_dir}")
            return
        
        # Get all files in directory
        files = [f for f in local_path.iterdir() if f.is_file()]
        
        if not files:
            print(f"No files found in {local_dir}")
            return
        
        print(f"\nFound {len(files)} file(s) to upload")
        print("-" * 50)
        
        success_count = 0
        for file_path in files:
            # Create dropbox path
            dropbox_file_path = f"{dropbox_dir}/{file_path.name}"
            if self.upload_file(str(file_path), dropbox_file_path):
                success_count += 1
        
        print("-" * 50)
        print(f"\nCompleted: {success_count}/{len(files)} files uploaded successfully")
    
    def list_folder(self, dropbox_path=""):
        """List contents of a Dropbox folder"""
        try:
            result = self.dbx.files_list_folder(dropbox_path)
            print(f"\nContents of '{dropbox_path or '/'}':")
            for entry in result.entries:
                print(f"  - {entry.name}")
        except ApiError as e:
            print(f"Error listing folder: {e}")


 
# OPTION 2: Or paste your token here (not recommended for production)
ACCESS_TOKEN ='sl.u.AGK9Ohq7aXLZvLjoO1mUmLQUrykYGJmtocbVxT2OWtt7xHHxD0m0cRhJ_jKLzSSyXzCdvFZeuWx6yhn9-DAz5pWv8-MlyN5AUerpouIzi_4giE1-TLpC3G5XVIizncgw7fefTvo0DYvok6mSLIftjOptczU1A7XQKwFm3gKf9UBYYAuihqbmHNqGeDlOP9kwOjcL9tPDVJeWPRkOAnUiZUxhd1cJiNphds_T0Dqb4Rp0OuCm19GZIE-kmgeXQ_2JbeaaYDTJikEiTXdwAv-jjMIi-1nmQ67thbLuL4TzYGZ-FTEdccOpJ_DbafJt6Dms5yxWzyXnOrk1SzUgoP3vynjSdDsemzdc5iBErWLCH-xMZF394bpqkXSLa0L2x58xUsaTmak0t5moWmOYdPlNrog8ri1xUORp5evK8ZwkFuN4WaRdW2x_j2uhNRyAewO3QGoG6hDWQHKkDCEyhR7TSFLhasSWerq_DvCtL3fifMKUuCxXz-MlrwLp-M0G77diuwwAZNv7H6Lc9MdSsfOogPvBHODU0EG3JiREKxOWeWiYLdC3GofnNIwikAHZYkUU7_TgyEfit7-4y2vRFEyOhkvNzlV9v4re2mMiOrl6Wal6upYf2v9zOhxDohvZrdbtRCimtCdKcJ-O4-VVHdeDoLik78JKQ4TfulHTOqT5hFqhgUivmPIXQoNvCK-cmixNn0pDvQB1TOXOXI_jGc_Nv6GZpcVx9Tfsx9-415L-pfttnzB_AxZSElLFDX0iWv_ihqkTbPHQS968Mn7kSIjCCvsiZshmUe5Bvcqqn4ySeyBxqZloQG3rV6Rumx3uzGDYVWoY8dwCjEO19H5_vfD_eBG8pi_R_YqJ74jCkAWi9bEEePVGkf09XTFgpn3pIP4JpEP7cb3ykEUh9gHnMqz_FFELn8lDVTNHwLP8Uh27ieTfeFXD11r2t41s9l1m_I1Ec3yv6xZVEkus7a1HnTslDvYjTtCuSe5cmMx3tKKE6qZ8FCsJM_G1Y6jDHcM6cgb4PkBgr4jJjU-J0sOaSBgALSCPFFfpiOzPUoHSs6RbNzz2uTbHnxkvbVm7EgvxOBY_wcCXqgsRlmbE9O2uuCY_qAHsYtYoSs7UUksy9MiuXRkSvZK8DElc6L4rYMqSC2i2BYmmpAN_1LyGULiM3opzKkM8XVRq2vVDXXuNpsZW-QEAT-RQL5HSWG2zZ_f-H05InLgW1_oJmGLgZwH69A6qoI09wUFNhzZkZO8lF6Dazuk0exZ3LZrAcGfCh3jQdFFpgkiqropc0CHMfZtxUhlU02UM6Em8gaKkIfNyVV8W2zjEeg'

# Initialize uploader
uploader = DropboxUploader(ACCESS_TOKEN)


Successfully connected to Dropbox!


In [32]:
uploader.upload_file('full_affiliations_data.csv', '/Kenny/orcid/full_affiliations_data.csv')

Uploading full_affiliations_data.csv (5529.45 MB) to /Kenny/orcid/full_affiliations_data.csv...
  Progress: 100.0%
✓ Successfully uploaded full_affiliations_data.csv


True

In [ ]:
uploader.upload_file('all_default10.csv', '/Kenny/orcid/all_default10.csv')
uploader.upload_file('all_default5.csv', '/Kenny/orcid/all_default5.csv')


Uploading all_default10.csv (6456.82 MB) to /Kenny/orcid/all_default10.csv...
  Progress: 6.8%